# supervised learning
## Regression

In [ ]:
!pip install optuna

Setting

In [ ]:
!pip install catboost

In [ ]:
!pip install jpholiday

In [ ]:
from common.supervised_learning import function
from common import regression

In [ ]:
# feature_selection range 50-150
n_features_to_select = 200

In [ ]:
pipelines = {}
pipelines = regression.setup_algorithm(pipelines)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import log_loss
evaluation_list = {'RMSE':'function.root_mean_squared_error',
                   'MAE':'mean_absolute_error',
                   'R2':'r2_score',
                   'log_loss':'log_loss',
                   'RMSLE':'function.root_mean_squared_log_error'}
evaluation_function_list = {'RMSE':function.root_mean_squared_error,
                            'MAE':mean_absolute_error,
                            'R2':r2_score,
                            'LOG_LOSS':log_loss,
                            'RMSLE':function.root_mean_squared_log_error}
options_evaluation = ['RMSE', 'MAE', 'R2', 'LOG_LOSS', 'RMSLE']

/Users/shin/anaconda_space/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [ ]:
options_algorithm = ['lightgbm', 'xgboost', 'catboost', 'tree', 'rsvc', 'logistic', 'rf', 'gb', 'mlp']
exception_algorithm_list = ['ols', 'ridge', 'knn', 'rsvc', 'gbr1', 'gbr2', 'mlp', 'logistic', 'catboost', 'xgboost', 'tree', 'rf']
exception_algorithm_list = ['ols', 'ridge', 'knn', 'rsvc', 'gbr1', 'gbr2', 'mlp', 'logistic', 'tree', 'rf']
for algorithm in exception_algorithm_list:
    if algorithm in pipelines:
        pipelines.pop(algorithm)
# Feature selection by RandomForestClassifier
feature_selection_rf_list = ['knn', 'logistic', 'rsvc', 'mlp']
feature_importances_algorithm_list = ['tree', 'rf', 'gb', 'lightgbm', 'xgboost', 'catboost']
# カテゴリ変数をリストで設定
ohe_columns = [
               'branch_id',
               'item_id',
               'item_category_id'
            ]
# カテゴリ変数をobject型で読み込むための準備
my_dtype = {
            'branch_id':object,
            'item_id':object,
            'item_category_id':object,
            'item_category_name':object,
            'sale_price':'float32',
            'sale_amount':'float32',
            }
id_label = ['index']
target_label = ['sale_amount']
drop_columns = ['yyyy-mm-dd']
drop_columns = []
train_file_name = input_data_dir + 'sales_history.csv'
addon_file_name = input_data_dir + 'item_categories.csv'
addon_file_name_ex = input_data_dir + 'category_names.csv'
test_file_name = input_data_dir + 'test.csv'
model_columns_file_name = output_data_dir + 'model_columns.csv'
score_columns_file_name = output_data_dir + 'score_columns.csv'
train_file_name = input_data_dir + 'sales_history.csv'
addon_file_name = input_data_dir + 'item_categories.csv'
addon_file_name_ex = input_data_dir + 'category_names.csv'
column_file_name = input_data_dir + 'column_names.json'
test_file_name = input_data_dir + 'test.csv'
# feature_selection range 50-100
n_features_to_select = 200
# ファイル出力拡張子
file_extention = 'csv'
is_header = False

In [ ]:
# ホールドアウト有無
is_holdout = False
# k-fold 法を利用するか
is_k_fold = True
# 
is_optuna = False
is_sklearn = False
# 目的変数をマイナスから0にするかどうか
is_target_minus_to_zero = True
is_imputation = True
is_one_hot_encoding = False
is_category_encoded = False
is_target_encoded  = False
is_feature_selection = False
is_test_to_train = False
isLog = False
is_gift = False 
is_console_game = False
# 1.97
is_game = False # 2.03
is_console_game_a = False   # 2.00
is_console_game_b = False   # 2.00
is_console_game_c = False   # 1.98
is_calendar = True

In [ ]:
def isHoliday(timestamp):
    import datetime
    import jpholiday
    if timestamp.weekday() >= 5 or jpholiday.is_holiday(timestamp):
        return 1
    else:
        return 0

In [ ]:
import json
with open(column_file_name, "r", encoding="utf-8") as f:
    d = json.load(f)
import pandas as pd
import numpy as np
def input_train_file(filename, my_dtype, id_label):
    # set data by role
    df = pd.read_csv(train_file_name, header=0, dtype=my_dtype)
    df = df.rename(columns=d)
    df_category = pd.read_csv(addon_file_name, header=0, dtype=my_dtype)
    df_category = df_category.rename(columns=d)
    df_train = pd.merge(df, df_category, on='item_id')
    del df, df_category
    import gc
    gc.collect()
    ymd= pd.to_datetime(df_train['yyyy-mm-dd'], errors='coerce')
    df_train["month"] = ymd.dt.strftime('%m').astype(int)
    #df_train = df_train[df_train["month"].astype(int) == 12]
    df_train = df_train.reset_index(drop=True)
    df_train["year_month"] = 12 * (ymd.dt.year.astype(int) - 2018) + df_train["month"].astype(int)

    #df_train = df_train[(df_train["year_month"].astype(int) == 22) | (df_train["month"].astype(int) == 12)]

    df_train = df_train.groupby(['year_month', 'month', 'branch_id', 'item_id', 'item_category_id', 'sale_price']).sum().reset_index()
    df_train = df_train.sort_values(['year_month'])
    df_train = df_train.reset_index(drop=True)
    for column in target_label:
        y = df_train[column]
    df_train = df_train.drop(target_label, axis=1)
    df_train = df_train.drop('sale_price', axis=1)
    X = df_train.reset_index(drop=True)
    del df_train
    import gc
    gc.collect()
    X.to_csv(output_data_dir + 'X.csv')
    print('欠損個数（数値変数の欠損補完前）:input_train_file', X.isnull().sum().sum())
    print(X.isnull().sum())
    return X, y

In [ ]:
def day_input_train_file(filename, my_dtype, item_ids=[]):
    from datetime import datetime
    import jpholiday
    df = pd.read_csv(train_file_name, header=0, dtype=my_dtype)
    df = df.rename(columns=d)
    df_category = pd.read_csv(addon_file_name, header=0, dtype=my_dtype)
    df_category = df_category.rename(columns=d)
    df_train = pd.merge(df, df_category, on='item_id')
    del df, df_category
    import gc
    gc.collect()

    df_train["timestamp"] = pd.to_datetime(df_train['yyyy-mm-dd'], utc=True)
    timestamp = pd.to_datetime(df_train['yyyy-mm-dd'], utc=True)
    df_train = df_train.groupby(['timestamp']).sum().reset_index()
    if is_calendar:
        df_train['count_day'] = 1
        df_train['count_holiday'] = timestamp.map(isHoliday)
    
    df_train["year_month"] = 12 * (df_train["timestamp"].dt.year.astype(int) - 2018) + df_train["timestamp"].dt.month.astype(int)
    df_train = df_train.groupby(['year_month']).sum().reset_index()
    df_train = df_train.drop('sale_price', axis=1)
    df_train = df_train.drop('branch_id', axis=1)
    df_train = df_train.drop('item_id', axis=1)
    df_train = df_train.drop('item_category_id', axis=1)
    df_train = df_train.drop('sale_amount', axis=1)
    #df_train = df_train.drop('item_category_name', axis=1)
    df_train = df_train.reset_index(drop=True)
    X = df_train.reset_index(drop=True)
    X.to_csv(output_data_dir + 'day_X.csv')
    print('欠損個数（数値変数の欠損補完前）:input_train_file', X.isnull().sum().sum())
    return X, y

In [ ]:
def input_test_file(filename, my_dtype, id_label):
    df_score = pd.read_csv(filename, header=0, dtype=my_dtype)
    df_score = df_score.rename(columns=d)
    df_score = df_score.reset_index(drop=True)
    df_score['month'] = 12
    df_score['year_month'] = 12 * 2
    df_category = pd.read_csv(addon_file_name, header=0, dtype=my_dtype)
    df_category = df_category.rename(columns=d)
    df_score = pd.merge(df_score, df_category, on='item_id')
    del df_category
    import gc
    gc.collect()
    X_s = df_score.drop(id_label, axis=1)
    X_s = X_s.reset_index(drop=True)
    return df_score, X_s

In [ ]:
import pandas as pd
import datetime as dt
import re
def transform_data(X:pd.core.series.Series):
    if is_gift:
        X['is_gift'] = 0
        m = X.category_name.str.contains('ギフト')
        X['is_gift'] = np.where(m, 1, 0)

    if is_console_game:
        X['is_console_game'] = 0
        m = X.category_name.str.contains('ゲーム機')
        X['is_console_game'] = np.where(m, 1, 0)

    if is_console_game_a:
        X['is_console_game_a'] = 0
        m = X.category_name.str.contains('ゲーム機A')
        X['is_console_game_a'] = np.where(m, 1, 0)

    if is_console_game_b:
        X['is_console_game_b'] = 0
        m = X.category_name.str.contains('ゲーム機B')
        X['is_console_game_b'] = np.where(m, 1, 0)

    if is_console_game_c:
        X['is_console_game_c'] = 0
        m = X.category_name.str.contains('ゲーム機C')
        X['is_console_game_c'] = np.where(m, 1, 0)

    if is_game:
        X['is_game'] = 0
        m = X.category_name.str.contains('ゲーム')
        X['is_game'] = np.where(m, 1, 0)

    if isLog:
        for column in log_columns:
            X['log_' + column] = np.log1p(X[column])
    
    #X["branch_item"] = str(X["branch"]) + '_' + str(X["item"])
    #X = function.one_hot_encoding(X, ['branch_item'])
    #X["item"] = X["item"] - X["item"].min() + 1
    #X["category"] = X["category"] - X["category"].min() + 1

    if is_category_encoded:
        for column in ohe_columns:
            X['frequency_encode_' + column] = function.convert_to_frequency_encode(X, column)
            X['label_encode_' + column] = function.convert_to_label_encode(X, column)
            X['count_encode_' + column] = function.convert_to_count_encode(X, column)
            X['label_count_encode_' + column] = function.convert_to_label_count_encode(X, column)
        '''
        if not is_target_encoded and not is_one_hot_encoding:
            for column in ohe_columns:
                X = X.drop(column, axis=1)
                X = X.reset_index(drop=True)
        '''
    for column in drop_columns:
        X = X.drop(column, axis=1)
    X = X.reset_index(drop=True)

    return X

In [ ]:
input_evaluation = function.choice(options_evaluation)

RadioButtons(options=('RMSE', 'MAE', 'R2', 'LOG_LOSS', 'RMSLE'), value='RMSE')

In [ ]:
X, y = input_train_file(train_file_name, my_dtype, id_label)

欠損個数（数値変数の欠損補完前）:input_train_file 0
year_month          0
month               0
branch_id           0
item_id             0
item_category_id    0
dtype: int64


In [ ]:
if is_calendar:
    day_X, y = day_input_train_file(train_file_name, my_dtype)
    X = pd.merge(day_X, X, on='year_month')

欠損個数（数値変数の欠損補完前）:input_train_file 0


In [ ]:
X = transform_data(X)

In [ ]:
df_s, X_s = input_test_file(test_file_name, my_dtype, id_label)
X_s = transform_data(X_s)

In [ ]:
if is_target_encoded:
    for column in ohe_columns:
        X[column] = X[column].fillna(-1)
        X_s[column] = X_s[column].fillna(-1)

    X, X_s = function.target_encode(X, y.values.ravel(), df_s, X_s, ohe_columns)
    if False:
        for column in ohe_columns:
            X = X.drop(column, axis=1)
            X = X.reset_index(drop=True)
            X_s = X_s.drop(column, axis=1)
            X_s = X_s.reset_index(drop=True)

In [ ]:
if is_one_hot_encoding:
    X_ohe = function.one_hot_encoding(X, ohe_columns)
else:
    for column in ohe_columns:
        X[column] = X[column].astype('category')
    X_ohe = X

X_ohe.to_csv(output_data_dir + "X_ohe.csv", index=False, header=True)
print('欠損個数（数値変数の欠損補完前）', X_ohe.isnull().sum().sum())
print('')
print(X_ohe.isnull().sum())

欠損個数（数値変数の欠損補完前） 0

year_month          0
count_day           0
count_holiday       0
month               0
branch_id           0
item_id             0
item_category_id    0
dtype: int64


In [ ]:
if is_imputation:
    X_ohe, X_ohe_columns = function.imputation(output_data_dir, model_columns_file_name, X_ohe)
else:
    X_ohe, X_ohe_columns = X_ohe, X_ohe.columns.values

In [ ]:
import gc
gc.collect()

44

In [ ]:
import time
start_at = time.time()
X_train, X_valid, y_train, y_valid = function.execute_feature_selection(output_data_dir,
                                                                        pipelines,
                                                                        feature_selection_rf_list,
                                                                        n_features_to_select, 
                                                                        X_ohe, y.values.ravel(), X_ohe_columns,
                                                                        is_holdout, is_optuna,
                                                                        is_feature_selection)
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

lightgbm
catboost
xgboost
Elapsed time: 0.00 [sec]


In [ ]:
import gc
gc.collect()

44

In [ ]:
if is_one_hot_encoding:
    categorical_feature = ohe_columns
elif is_target_encoded and not is_category_encoded:
    categorical_feature = []
else:
    categorical_feature = ohe_columns
import time
start_at = time.time()
X_train, X_valid, y_train, y_valid = regression.train_model(output_data_dir, pipelines,
                                                                X_train, X_valid, y_train, y_valid,
                                                                input_evaluation.value, is_holdout,
                                                                is_optuna, is_sklearn, 
                                                                categorical_feature=categorical_feature
                                                            )
end_at = time.time()
print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))

lightgbm


/Users/shin/anaconda_space/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/shin/anaconda_space/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/shin/anaconda_space/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['branch_id', 'item_category_id', 'item_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


catboost
normal
Learning rate set to 0.020807
0:	learn: 4.1610781	total: 129ms	remaining: 21m 31s
1:	learn: 4.1565880	total: 224ms	remaining: 18m 38s
2:	learn: 4.1521943	total: 286ms	remaining: 15m 52s
3:	learn: 4.1477009	total: 365ms	remaining: 15m 11s
4:	learn: 4.1434204	total: 431ms	remaining: 14m 21s
5:	learn: 4.1392303	total: 490ms	remaining: 13m 35s
6:	learn: 4.1353329	total: 547ms	remaining: 13m
7:	learn: 4.1316057	total: 624ms	remaining: 12m 59s
8:	learn: 4.1281880	total: 690ms	remaining: 12m 45s
9:	learn: 4.1246730	total: 757ms	remaining: 12m 36s
10:	learn: 4.1209920	total: 813ms	remaining: 12m 18s
11:	learn: 4.1178172	total: 893ms	remaining: 12m 23s
12:	learn: 4.1144306	total: 976ms	remaining: 12m 30s
13:	learn: 4.1112310	total: 1.07s	remaining: 12m 46s
14:	learn: 4.1082017	total: 1.19s	remaining: 13m 9s
15:	learn: 4.1050429	total: 1.25s	remaining: 12m 57s
16:	learn: 4.1024116	total: 1.3s	remaining: 12m 45s
17:	learn: 4.0996861	total: 1.36s	remaining: 12m 37s
18:	learn: 4.097

KeyboardInterrupt: ignored

In [ ]:
regression.display_evaluation(is_holdout, output_data_dir, pipelines, evaluation_function_list,
                                  input_evaluation, X_train, y_train, X_valid, y_valid
                                #, is_optuna,
                                 # is_sklearn
                              )

In [ ]:
input_algorithm = function.choice(options_algorithm)

In [ ]:
import datetime
from datetime import timezone
def main():
    algorithm_name = input_algorithm.value
    #df_s, X_s = input_test_file(test_file_name, my_dtype, id_label)
    #X_s = transform_data(X_s)
    #X_s.to_csv(out_put_data_dir + "X_s.csv", index=False, header=True)
    if is_one_hot_encoding:
        X_ohe_s = function.one_hot_encoding(X_s, ohe_columns)
    elif not is_target_encoded and not is_category_encoded:
        for column in ohe_columns:
            X_s[column] = X_s[column].astype('category')
        X_ohe_s = X_s
    else:
        X_ohe_s = X_s
    #X_ohe_s.to_csv(out_put_data_dir + "X_ohe_s.csv", index=False, header=True)
    X_predicted = function.preprocessing(output_data_dir, model_columns_file_name, score_columns_file_name,
                                         algorithm_name, X_ohe,
                                         X_ohe_s, is_imputation, is_feature_selection)
    #display(X_predicted)
    predict = regression.scoring(output_data_dir, algorithm_name, X_predicted);
    #display(predict)
    #predict = np.expm1(predict)
    if is_target_minus_to_zero:
        predict = [i if i > 0 else 0 for i in predict]
    function.output_file(output_data_dir, n_features_to_select, target_label, df_s, id_label,
                         predict, algorithm_name, file_extention, header=is_header)
    print(input_evaluation.value + ' selected')
    print(algorithm_name + ' selected')
    if algorithm_name in feature_importances_algorithm_list:
        if (is_optuna or not is_sklearn) and algorithm_name == 'lightgbm':
            ;
        else:
            feature_importances = pipelines[algorithm_name]['est'].feature_importances_
            feature_importances = pd.Series(feature_importances, index=X_predicted.columns.values.tolist())
            sorted_feature_importances = sorted(feature_importances.items(), key=lambda x:-x[1])
            sorted_feature_importances = pd.DataFrame(sorted_feature_importances)
            sorted_feature_importances.to_csv(output_data_dir + 'feature_importances_' + algorithm_name
                                              + '_' + datetime.datetime.now(timezone.utc).strftime('%Y%m%d%H%M%S') + '.csv', index=False)
            display(sorted_feature_importances)

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
if is_k_fold:
    import time
    start_at = time.time()
    # https://www.366service.com/jp/qa/0cdf422091e871fc451ef66a70f0550d
    from sklearn.metrics import make_scorer
    scorer = make_scorer(evaluation_function_list[input_evaluation.value], greater_is_better=True)
    from sklearn.model_selection import KFold
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import TimeSeriesSplit
    #kf = KFold(n_splits=5, shuffle=True, random_state=0)
    #kf = StratifiedKFold(n_splits=8, shuffle=True, random_state=0)
    kf = TimeSeriesSplit(n_splits=5)
    regression.cross_validatior(kf, scorer, output_data_dir,
                                    n_features_to_select, pipelines, input_evaluation, X_ohe, y.values.ravel())
    end_at = time.time()
    print("Elapsed time: {:.2f} [sec]".format(end_at - start_at))